In [1]:
# import the necessary packages
import tensorflow as tf
import pickle
from  util import  plotUtil
from modelsRepo import modelsFactory
from modelEvaluator import ModelEvaluator
from  util import  plotUtil  
from  util import  helper  
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from imutils import paths
import numpy as np
import argparse
import random
import cv2
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical

[INFO] matplotlib BACKEND IS Qt5Agg


Using TensorFlow backend.


In [19]:
width=224
height=224
EPOCHS =2
datasetDir="SportsClassification"
networkID="Resnet50"
channels=3

In [3]:
def predictBinaryValue(probs,threshold=0.5):
	y_pred=[]
	for prob in probs:
		if (prob>threshold):
			y_pred.append(1)
		else:	
			y_pred.append(0)
	return np.asarray(y_pred)

In [4]:
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

In [5]:
root_dir="datasets"
base_dir = os.path.join(root_dir,datasetDir)
# initial learning rate, and batch size

INIT_LR = 1e-3
BS = 32

In [6]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []
train_labels_dir=[]
folders=get_immediate_subdirectories(os.path.join(root_dir,datasetDir))
print("[INFO] Images  folder detected")

[INFO] loading images...
[INFO] Images  folder detected


In [7]:
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(base_dir)))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:

	# extract the class label from the image path and update the labels list
	label = imagePath.split(os.path.sep)[-2]


	# load the image, pre-process it, and store it in the data list
	print("[INFO] Reading image from path: {}".format(imagePath))
	
	if(channels==3):
		image = cv2.imread(imagePath)
	else:
		image = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
	
	image = cv2.resize(image, (width,height))
	image = img_to_array(image)	
	labels.append(label)
	#print(labels)
	data.append(image)


[INFO] Reading image from path: datasets/SportsClassification/volleyball/00000516.jpg
[INFO] Reading image from path: datasets/SportsClassification/swimming/00000187.png
[INFO] Reading image from path: datasets/SportsClassification/weight_lifting/00000274.jpg
[INFO] Reading image from path: datasets/SportsClassification/wwe/00000232.jpg
[INFO] Reading image from path: datasets/SportsClassification/motogp/00000206.jpg
[INFO] Reading image from path: datasets/SportsClassification/cricket/00000169.jpg
[INFO] Reading image from path: datasets/SportsClassification/swimming/00000635.jpg
[INFO] Reading image from path: datasets/SportsClassification/tennis/00000254.jpg
[INFO] Reading image from path: datasets/SportsClassification/chess/00000162.jpg
[INFO] Reading image from path: datasets/SportsClassification/volleyball/00000379.jpg
[INFO] Reading image from path: datasets/SportsClassification/wrestling/00000606.jpg
[INFO] Reading image from path: datasets/SportsClassification/basketball/00000

[INFO] Reading image from path: datasets/SportsClassification/kabaddi/00000433.jpg
[INFO] Reading image from path: datasets/SportsClassification/chess/00000227.jpg
[INFO] Reading image from path: datasets/SportsClassification/swimming/00000349.jpg
[INFO] Reading image from path: datasets/SportsClassification/fencing/00000211.jpg
[INFO] Reading image from path: datasets/SportsClassification/kabaddi/00000120.jpg
[INFO] Reading image from path: datasets/SportsClassification/boxing/00000310.jpg
[INFO] Reading image from path: datasets/SportsClassification/badminton/00000089.jpg
[INFO] Reading image from path: datasets/SportsClassification/motogp/00000527.jpg
[INFO] Reading image from path: datasets/SportsClassification/badminton/00000597.jpg
[INFO] Reading image from path: datasets/SportsClassification/chess/00000113.jpg
[INFO] Reading image from path: datasets/SportsClassification/kabaddi/00000335.jpg
[INFO] Reading image from path: datasets/SportsClassification/swimming/00000137.jpg
[INF

[INFO] Reading image from path: datasets/SportsClassification/football/00000608.jpg
[INFO] Reading image from path: datasets/SportsClassification/table_tennis/00000162.jpg
[INFO] Reading image from path: datasets/SportsClassification/table_tennis/00000696.jpg
[INFO] Reading image from path: datasets/SportsClassification/weight_lifting/00000263.jpg
[INFO] Reading image from path: datasets/SportsClassification/chess/00000157.jpg
[INFO] Reading image from path: datasets/SportsClassification/wrestling/00000216.jpeg
[INFO] Reading image from path: datasets/SportsClassification/baseball/00000039.jpg
[INFO] Reading image from path: datasets/SportsClassification/cricket/00000261.jpg
[INFO] Reading image from path: datasets/SportsClassification/basketball/00000226.jpg
[INFO] Reading image from path: datasets/SportsClassification/boxing/00000520.jpg
[INFO] Reading image from path: datasets/SportsClassification/tennis/00000419.jpg
[INFO] Reading image from path: datasets/SportsClassification/kaba

[INFO] Reading image from path: datasets/SportsClassification/boxing/00000382.jpg
[INFO] Reading image from path: datasets/SportsClassification/volleyball/00000168.jpg
[INFO] Reading image from path: datasets/SportsClassification/wwe/00000138.jpg
[INFO] Reading image from path: datasets/SportsClassification/wrestling/00000018.jpg
[INFO] Reading image from path: datasets/SportsClassification/tennis/00000089.jpg
[INFO] Reading image from path: datasets/SportsClassification/boxing/00000724.jpg
[INFO] Reading image from path: datasets/SportsClassification/badminton/00000456.jpg
[INFO] Reading image from path: datasets/SportsClassification/wrestling/00000415.jpg
[INFO] Reading image from path: datasets/SportsClassification/boxing/00000708.jpg
[INFO] Reading image from path: datasets/SportsClassification/table_tennis/00000742.jpg
[INFO] Reading image from path: datasets/SportsClassification/motogp/00000674.jpg
[INFO] Reading image from path: datasets/SportsClassification/ice_hockey/00000203.

[INFO] Reading image from path: datasets/SportsClassification/wrestling/00000063.jpg
[INFO] Reading image from path: datasets/SportsClassification/table_tennis/00000612.jpg
[INFO] Reading image from path: datasets/SportsClassification/swimming/00000260.jpg
[INFO] Reading image from path: datasets/SportsClassification/weight_lifting/00000469.jpg
[INFO] Reading image from path: datasets/SportsClassification/gymnastics/00000762.JPG
[INFO] Reading image from path: datasets/SportsClassification/table_tennis/00000281.jpg
[INFO] Reading image from path: datasets/SportsClassification/boxing/00000274.jpg
[INFO] Reading image from path: datasets/SportsClassification/volleyball/00000078.jpg
[INFO] Reading image from path: datasets/SportsClassification/table_tennis/00000368.png
[INFO] Reading image from path: datasets/SportsClassification/cricket/00000157.jpg
[INFO] Reading image from path: datasets/SportsClassification/swimming/00000602.jpg
[INFO] Reading image from path: datasets/SportsClassific

...

[INFO] Reading image from path: datasets/SportsClassification/kabaddi/00000040.jpeg
[INFO] Reading image from path: datasets/SportsClassification/badminton/00000409.jpg
[INFO] Reading image from path: datasets/SportsClassification/fencing/00000083.jpg
[INFO] Reading image from path: datasets/SportsClassification/badminton/00000335.jpg
[INFO] Reading image from path: datasets/SportsClassification/wrestling/00000253.JPG
[INFO] Reading image from path: datasets/SportsClassification/formula1/00000446.jpg
[INFO] Reading image from path: datasets/SportsClassification/football/00000155.jpg
[INFO] Reading image from path: datasets/SportsClassification/basketball/00000006.jpg
[INFO] Reading image from path: datasets/SportsClassification/cricket/00000557.jpg
[INFO] Reading image from path: datasets/SportsClassification/shooting/00000561.jpg
[INFO] Reading image from path: datasets/SportsClassification/baseball/00000345.jpg
[INFO] Reading image from path: datasets/SportsClassification/formula1/00

[INFO] Reading image from path: datasets/SportsClassification/ice_hockey/00000518.jpg
[INFO] Reading image from path: datasets/SportsClassification/kabaddi/00000011.jpg
[INFO] Reading image from path: datasets/SportsClassification/swimming/00000006.jpg
[INFO] Reading image from path: datasets/SportsClassification/wrestling/00000535.jpg
[INFO] Reading image from path: datasets/SportsClassification/hockey/00000081.jpg
[INFO] Reading image from path: datasets/SportsClassification/fencing/00000224.jpg
[INFO] Reading image from path: datasets/SportsClassification/boxing/00000051.jpg
[INFO] Reading image from path: datasets/SportsClassification/shooting/00000501.jpg
[INFO] Reading image from path: datasets/SportsClassification/motogp/00000643.jpg
[INFO] Reading image from path: datasets/SportsClassification/ice_hockey/00000228.jpg
[INFO] Reading image from path: datasets/SportsClassification/volleyball/00000653.jpg
[INFO] Reading image from path: datasets/SportsClassification/volleyball/0000

[INFO] Reading image from path: datasets/SportsClassification/motogp/00000466.jpg
[INFO] Reading image from path: datasets/SportsClassification/kabaddi/00000097.jpg
[INFO] Reading image from path: datasets/SportsClassification/wwe/00000069.jpg
[INFO] Reading image from path: datasets/SportsClassification/basketball/00000368.jpg
[INFO] Reading image from path: datasets/SportsClassification/badminton/00000530.jpg
[INFO] Reading image from path: datasets/SportsClassification/kabaddi/00000133.jpg
[INFO] Reading image from path: datasets/SportsClassification/badminton/00000507.jpg
[INFO] Reading image from path: datasets/SportsClassification/fencing/00000672.png
[INFO] Reading image from path: datasets/SportsClassification/ice_hockey/00000739.jpg
[INFO] Reading image from path: datasets/SportsClassification/baseball/00000627.jpg
[INFO] Reading image from path: datasets/SportsClassification/tennis/00000614.jpg
[INFO] Reading image from path: datasets/SportsClassification/volleyball/00000387.

[INFO] Reading image from path: datasets/SportsClassification/chess/00000462.jpeg
[INFO] Reading image from path: datasets/SportsClassification/tennis/00000624.jpg
[INFO] Reading image from path: datasets/SportsClassification/badminton/00000257.jpg
[INFO] Reading image from path: datasets/SportsClassification/gymnastics/00000041.JPG
[INFO] Reading image from path: datasets/SportsClassification/wrestling/00000611.jpg
[INFO] Reading image from path: datasets/SportsClassification/basketball/00000393.jpg
[INFO] Reading image from path: datasets/SportsClassification/wrestling/00000305.JPG
[INFO] Reading image from path: datasets/SportsClassification/swimming/00000688.jpg
[INFO] Reading image from path: datasets/SportsClassification/wrestling/00000263.jpg
[INFO] Reading image from path: datasets/SportsClassification/cricket/00000516.jpg
[INFO] Reading image from path: datasets/SportsClassification/table_tennis/00000073.jpg
[INFO] Reading image from path: datasets/SportsClassification/volleyb

KeyboardInterrupt: 

In [8]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [9]:
#lb.classes_  will be  the labels with the same order in one hot vector--->. label = lb.classes_[i]
lb = LabelBinarizer() 
labels = lb.fit_transform(labels)  #Binary targets transform to a column vector. otherwise one hot vector, you can also use from keras.utils.to_categorical to  perform one-hot encoding on the labels
numOfOutputs=len(lb.classes_)
#print(lb.classes_)

In [10]:
if (numOfOutputs==2):  #Binary problem
	numOfOutputs=1  # use only 1 neuron in last layer

print("[INFO] Training with the following {} classes {}".format(numOfOutputs ,lb.classes_ ))   

[INFO] Training with the following 22 classes ['badminton' 'baseball' 'basketball' 'boxing' 'chess' 'cricket' 'fencing'
 'football' 'formula1' 'gymnastics' 'hockey' 'ice_hockey' 'kabaddi'
 'motogp' 'shooting' 'swimming' 'table_tennis' 'tennis' 'volleyball'
 'weight_lifting' 'wrestling' 'wwe']


In [12]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)
numPfSamples,imgWidth,imgHeight,numOfchannels=trainX.shape
print("[INFO] Dataset of trainData shape {}".format(trainX.shape))
print("[INFO] Dataset of trainLabels shape {}".format(testX.shape))
print("[INFO] Dataset of testData shape {}".format(trainY.shape))
print("[INFO] Dataset of testLabels shape {}".format(testY.shape))

[INFO] Dataset of trainData shape (2192, 224, 224, 3)
[INFO] Dataset of trainLabels shape (731, 224, 224, 3)
[INFO] Dataset of testData shape (2192, 22)
[INFO] Dataset of testLabels shape (731, 22)


In [13]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [15]:
# initialize the model
print("[INFO] compiling model...")
#model = LeNet.build(width=28, height=28, depth=3, classes=1)
#model=modelsFactory.ModelCreator(numOfOutputs,imgWidth,imgHeight,"Resnet50").model
model=modelsFactory.ModelCreator(numOfOutputs,width,height,channels=channels,networkID=networkID).model
model.summary()

[INFO] compiling model...


/Users/walidahmed/anaconda2/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[INFO]  Resnet50 created
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         con

In [17]:
opt = tf.keras.optimizers.Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
if(numOfOutputs==1):
	model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
else:
	model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


In [ ]:
# train the network
history = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

Epoch 1/2
47/69 [===================>..........] - ETA: 32:26 - loss: 2.8903 - acc: 0.2041

In [ ]:
# save the model to disk
fileNameToSaveModel="{}_binaryClassifier.keras2".format(datasetDir)
fileNameToSaveModel=os.path.join("Results",fileNameToSaveModel)
model.save(fileNameToSaveModel)
print("[INFO] Model saved  to file {}".format(fileNameToSaveModel))

In [ ]:
# serialize the label binarizer to disk
fileNameToSaveLabels=datasetDir+"_labels.pkl"
fileNameToSaveLabels=os.path.join("Results",fileNameToSaveLabels)
f = open(fileNameToSaveLabels, "wb")
f.write(pickle.dumps(lb.classes_))
f.close()
print("[INFO] Labels saved  to file {}".format(fileNameToSaveLabels))

In [ ]:
if(numOfOutputs==1):
	labelsDict=dict()
	labelsDict[lb.classes_[0]]=0
	labelsDict[lb.classes_[1]]=1
	print(labelsDict)

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
if (numOfOutputs==1):
	y_true=testY
	y_pred=predictBinaryValue(predictions)
else:	
	y_true=testY.argmax(axis=1)
	y_pred=predictions.argmax(axis=1)

In [ ]:
print(classification_report(y_true,y_pred, target_names=lb.classes_))
cm=confusion_matrix(y_true, y_pred)
helper.print_cm(cm,lb.classes_)

In [ ]:
plotUtil.plotAccuracyAndLossesonSameCurve(history)
plotUtil.plotAccuracyAndLossesonSDifferentCurves(history)

In [ ]:
# Plot non-normalized confusion matrix
helper.plot_print_confusion_matrix(y_true, y_pred, classes=lb.classes_,dataset=datasetDir,title=datasetDir+ '_Confusion matrix, without normalization') 